# 🏰 Chess Bot Arena
Play chess against your fine-tuned MLX model!

In [ ]:
# Cell 1: Setup and Imports
import chess
import chess.svg
from IPython.display import SVG, display, clear_output
from mlx_lm import load, generate
import random
import re

print("🚀 Loading chess bot...")
try:
    model, tokenizer = load("chess_bot_fused")
    print("✅ Chess bot loaded successfully!")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("Make sure 'chess_bot_fused' directory exists")

In [ ]:
# Cell 2: Helper Functions
def get_bot_move(board):
    """Get the bot's move for the current position"""
    fen = board.fen()
    prompt = f"FEN: {fen}\nMove:"
    
    try:
        # Generate move
        response = generate(
            model, 
            tokenizer, 
            prompt=prompt, 
            max_tokens=10
        )
        
        # Extract the move from response
        move_text = response.strip()
        if move_text.startswith(prompt):
            move_text = move_text[len(prompt):].strip()
        
        # Clean up the move text (remove extra tokens)
        move_text = move_text.split()[0] if move_text.split() else ""
        
        # Try to parse the move
        move = board.parse_san(move_text)
        return move, move_text
        
    except Exception as e:
        print(f"🤖 Bot had trouble with move '{move_text}': {e}")
        # Fallback to a random legal move
        legal_moves = list(board.legal_moves)
        if legal_moves:
            fallback_move = random.choice(legal_moves)
            return fallback_move, board.san(fallback_move) + " (random)"
        return None, None

def display_board(board, last_move=None):
    """Display the chess board with optional last move highlighted"""
    if last_move:
        svg = chess.svg.board(board=board, size=400, lastmove=last_move)
    else:
        svg = chess.svg.board(board=board, size=400)
    display(SVG(svg))

def get_material_balance(board):
    """Calculate material balance"""
    piece_values = {
        chess.PAWN: 1, chess.KNIGHT: 3, chess.BISHOP: 3,
        chess.ROOK: 5, chess.QUEEN: 9, chess.KING: 0
    }
    
    white_material = sum(piece_values[piece.piece_type] 
                        for piece in board.piece_map().values() 
                        if piece.color == chess.WHITE)
    black_material = sum(piece_values[piece.piece_type] 
                        for piece in board.piece_map().values() 
                        if piece.color == chess.BLACK)
    
    return white_material - black_material

print("✅ Helper functions loaded!")

In [ ]:
# Cell 3: Initialize New Game
board = chess.Board()
game_history = []
move_count = 0

print("🎯 New Chess Game Started!")
print("You are playing as White ♔")
print("Bot is playing as Black ♚")
print("\nEnter moves in algebraic notation: e4, Nf3, O-O, etc.")
print("Type 'help' to see legal moves")
print("=" * 50)

display_board(board)
print(f"\n📋 Current FEN: {board.fen()}")
print(f"🎲 Legal moves: {len(list(board.legal_moves))}")

In [ ]:
# Cell 4: Make Your Move (Run this cell to play)

if board.is_game_over():
    print("🏁 Game is over! Run Cell 3 to start a new game.")
else:
    if board.turn == chess.WHITE:
        print(f"\n🎯 Move {board.fullmove_number}: Your turn (White ♔)")
        
        # Show some legal moves as hints
        legal_moves = list(board.legal_moves)
        legal_sans = [board.san(move) for move in legal_moves[:8]]  # Show first 8
        print(f"💡 Some legal moves: {', '.join(legal_sans)}{'...' if len(legal_moves) > 8 else ''}")
        
        # Get user input
        human_move = input("\n📝 Enter your move: ").strip()
        
        if human_move.lower() == 'help':
            all_legal = [board.san(move) for move in legal_moves]
            print(f"\n📋 All legal moves: {', '.join(all_legal)}")
        
        elif human_move:
            try:
                move = board.parse_san(human_move)
                board.push(move)
                game_history.append(f"{board.fullmove_number-1}. {human_move}")
                
                print(f"✅ You played: {human_move}")
                display_board(board, last_move=move)
                
                # Check game status after your move
                if board.is_checkmate():
                    print("🎉 Checkmate! You won!")
                elif board.is_stalemate():
                    print("🤝 Stalemate - it's a draw!")
                elif board.is_check():
                    print("⚠️ Check! Bot is in check.")
                
                # Bot's turn (if game not over)
                if not board.is_game_over():
                    print(f"\n🤖 Move {board.fullmove_number}: Bot's turn (Black ♚)")
                    print("🤔 Bot is thinking...")
                    
                    bot_move, bot_move_text = get_bot_move(board)
                    if bot_move:
                        board.push(bot_move)
                        game_history.append(bot_move_text)
                        
                        print(f"🤖 Bot played: {bot_move_text}")
                        display_board(board, last_move=bot_move)
                        
                        # Check game status after bot's move
                        if board.is_checkmate():
                            print("💀 Checkmate! Bot won!")
                        elif board.is_stalemate():
                            print("🤝 Stalemate - it's a draw!")
                        elif board.is_check():
                            print("⚠️ Check! You are in check.")
                        
                        # Show position info
                        material = get_material_balance(board)
                        if material > 0:
                            print(f"📊 Material: White +{material}")
                        elif material < 0:
                            print(f"📊 Material: Black +{abs(material)}")
                        else:
                            print(f"📊 Material: Equal")
                            
                    else:
                        print("❌ Bot couldn't generate a valid move!")
                        
            except ValueError:
                print(f"❌ Invalid move: '{human_move}'")
                print("💡 Try again with proper notation (e.g., e4, Nf3, O-O)")
    
    else:
        print("🤖 It's the bot's turn. This shouldn't happen - run Cell 3 to reset.")

# Show current game state
if not board.is_game_over():
    print(f"\n📋 Current FEN: {board.fen()}")
    print(f"🎲 Legal moves available: {len(list(board.legal_moves))}")
else:
    print(f"\n🏁 Final result: {board.result()}")
    print(f"📋 Final FEN: {board.fen()}")
    print(f"📜 Game moves: {' '.join(game_history)}")

In [ ]:
# Cell 5: Game Analysis & Utilities

print("📊 GAME ANALYSIS")
print("=" * 30)

print(f"🎯 Turn: {'White' if board.turn else 'Black'}")
print(f"🔢 Move number: {board.fullmove_number}")
print(f"🎲 Legal moves: {len(list(board.legal_moves))}")
print(f"👑 In check: {'Yes' if board.is_check() else 'No'}")

material_balance = get_material_balance(board)
if material_balance > 0:
    print(f"📊 Material: White +{material_balance}")
elif material_balance < 0:
    print(f"📊 Material: Black +{abs(material_balance)}")
else:
    print(f"📊 Material: Equal")

if board.is_game_over():
    result = board.result()
    if result == "1-0":
        print("🎉 White (You) won!")
    elif result == "0-1":
        print("🤖 Black (Bot) won!")
    else:
        print("🤝 Draw!")

print(f"\n📜 Move history: {' '.join(game_history)}")
print(f"\n📋 Current FEN:\n{board.fen()}")

# Show current board
display_board(board)

In [ ]:
# Cell 6: Test Bot on Custom Position

print("🔬 TEST BOT ON CUSTOM POSITION")
print("=" * 35)

# You can change this FEN to any position you want to test
test_fen = "rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq e3 0 1"  # After 1.e4

print(f"Testing position: {test_fen}")

try:
    test_board = chess.Board(test_fen)
    print(f"Turn: {'White' if test_board.turn else 'Black'}")
    
    display_board(test_board)
    
    # Get bot's move for this position
    print("\n🤖 Bot's recommended move:")
    bot_move, bot_text = get_bot_move(test_board)
    
    if bot_move:
        print(f"✅ Bot suggests: {bot_text}")
        
        # Show position after bot's move
        test_board.push(bot_move)
        print("\nPosition after bot's move:")
        display_board(test_board, last_move=bot_move)
    else:
        print("❌ Bot couldn't find a move")
        
except ValueError as e:
    print(f"❌ Invalid FEN: {e}")

print("\n💡 To test other positions, change the 'test_fen' variable and re-run this cell")

In [ ]:
# Cell 7: Reset Game (Quick restart)

print("🔄 RESETTING GAME...")

board = chess.Board()
game_history = []
move_count = 0

print("✅ New game ready!")
print("🎯 You are White ♔, Bot is Black ♚")
print("📝 Go to Cell 4 to make your first move!")

display_board(board)